In [ ]:
import os
import numpy as np
import msmrd2
import msmrd2.visualization as msmrdvis
from msmrd2 import continuousTimeMarkovStateModel as ctmsm
from msmrd2.potentials import dipole
from msmrd2.integrators import overdampedLangevinMarkovSwitch as odLangevinMS

In [ ]:
# Define continuous-time MSM
MSMtype = 1
ratematrix = np.array([[-3.0,3.0],[1.5,-1.5]])
seed = 0 # Seed = -1 used random device as seed
markovModel = ctmsm(MSMtype, ratematrix, seed)
Dlist = np.array([0.0, 0.0])
Drotlist = np.array([1.0, 1.0])
markovModel.setD(Dlist)
markovModel.setDrot(Drotlist)

In [ ]:
# Particle definition
orientation = np.array([1,0,0,0])
particlelist = []
# Create a grid of particles
for i in range(6):
    for j in range(6): 
        position = np.array([i-2.5,j-2.5,0])
        state = np.random.randint(0,2)
        part = msmrd2.particleMS(MSMtype, state, Dlist[state], Drotlist[state], position, orientation)   
        orientVector = np.random.normal(0,1,3)
        orientVector = orientVector/np.linalg.norm(orientVector)
        part.setOrientVector(orientVector)
        particlelist.append(part)
# Define list object that can be read by pybound functions
partlist = msmrd2.integrators.particleMSList(particlelist)

In [ ]:
# Over-damped Langevin integrator with Markovian Switch definition
dt = 0.001
seed = -1 # Seed = -1 used random device as seed
bodytype = "rod"
rotation = True
integrator = odLangevinMS(markovModel, dt, seed, bodytype, rotation)

In [ ]:
# Define and set potential
scalefactor = 30.0
Efieldvector = np.array([0,1,0])
potentialDipole = dipole(scalefactor, Efieldvector)
integrator.setExternalRodPotential(potentialDipole)

In [ ]:
# Integrate the particles and save to .dat to plot into VMD
datafile  = open('../data/vmd/odLangevinDipole.xyz', 'w')
timeIters = 10
prevstate = [None] * len(partlist)
for i in range(timeIters):
    #datafile  = open('odLangevinDipole.xyz', 'w')
    datafile.write(str(2*len(partlist)) + '\n')
    datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if part.state != prevstate[j] and prevstate[j] != None:
            part.setOrientVector(-part.orientvector)
        if part.state == 0:
            v0 = part.position
            v1 = v0 + 0.35*part.orientvector
            v2 = v0 - 0.35*part.orientvector
            datafile.write('type_0' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_0' + ' ' + ' '.join(map(str, v2)) + '\n')
        elif part.state == 1:
            v0 = part.position
            v1 = v0 + 0.35*part.orientvector
            v2 = v0 - 0.35*part.orientvector
            datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
        prevstate[j] = 1*part.state
    integrator.integrate(partlist)
datafile.close()

In [ ]:
# Generate tcl for plotting all frames needs -1 (Note vmd cannot change colors when state 
# changes. Check dipole script in vmd_movies for this). Load using vmd -e *.tcl
msmrdvis.generateTCL_dipole(-1,tclfname = "../data/vmd/odLangevinDipole2vmd.tcl")

In [ ]:
# Integrate the particle and print data
timeIters = 100
part1list = msmrd2.integrators.particleMSList([part])
print('{:<10s}{:<10s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position 1", "Orientation 1"))
for i in range(timeIters):
    print('{:<10d}{:<10f}{:<40s}{:<40s}'.format(i, integrator.clock, str(part1list[0].position), str(part1list[0].orientation)))
    integrator.integrate(part1list)